In [1]:
import pandas as pd
import numpy as np
import openpyxl
import pyarrow as pa
import pyarrow.parquet as pq
import glob
import os
from pandas.io.formats.excel import ExcelFormatter
from pandas.io import gbq
from openpyxl.workbook import Workbook
from shutil import copyfile
from pathlib import Path

In [2]:
old_path = "D:\\EcoBee\\New Data\\June Analysis\\Updated QC 3"
old_parquet_files = glob.glob(os.path.join(old_path, "*.parquet"))
new_path = "D:\\EcoBee\\New Data\\June Analysis\\Updated QC 4"
new_parquet_files = glob.glob(os.path.join(new_path, "*.parquet"))

In [3]:
    for file in old_parquet_files:
        df = (pd.read_parquet(file))
        filepath =  new_path + '\\'+ os.path.basename(file)     
        selected_columns = ['no_of_sensors', 'number_of readings', 'average_occupancy']  # Replace with the actual column names you want to select
        Hourly_Average_df = df[selected_columns].copy()
        selected_columns = [ 'average_occupancy']  # Replace with the actual column names you want to select
        HA_df = df[selected_columns].copy()
        ungrouped_HA = pd.pivot_table(HA_df, index='hour', columns='date')

        # Reset the column index to remove the multi-index structure
        ungrouped_HA.columns = ungrouped_HA.columns.droplevel()

        # Reset the DataFrame index to restore the original index
        ungrouped_HA.reset_index(inplace=True)
        
        ungrouped_HA.drop(ungrouped_HA.columns[0], axis=1, inplace =True)
        quartiles = ungrouped_HA.quantile([0.25, 0.5, 0.75, 1], axis=1)

        # Create a new DataFrame with quartile values
        Metric_df = pd.DataFrame(quartiles.values.T, columns=['Q1', 'Q2', 'Q3', 'Q4'])
        Metric_df['Average']= ungrouped_HA.mean(axis=1)
        
        Metric_df.to_parquet(filepath)
        


KeyboardInterrupt: 